# Compute the SNR of pilot data

## Goals:
1. Import the preprocessed data from npz and json files
2. Calculate the Signal-to-Noise Ratio (SNR) for each epoch (for use in stats)
3. Export the SNR


# Import Libraries

In [1]:
# Standard libraries
import json
import numpy as np
import pandas as pd
import scipy.signal as signal
import matplotlib.pyplot as plt

# Custom libraries
from Functions import processing

%matplotlib qt

# Import Epoched Data and Settings

In [2]:
# Load list of files to import
files = [  
    "sub-P001_ses-S001_task-T1_run-001_eeg"   
]

# Get unique subject IDs
subject_ids = [file.split('_')[0] for file in files]
unique_subject_ids = list(set(subject_ids))

# Preallocate variables to store EEG data and settings
eeg_epochs = [None] * len(files)
settings = [None] * len(files)
rs_open_data = [None] * len(files)

# Import data
for f, file in enumerate(files):
    # Import EEG data, since it is stored in a compressed numpy file (.npz) we need to use the np.load function 
    loaded_data = np.load(f"Data\\Masters_testing\\{file}.npz", allow_pickle=True)

    # Access the data for each stimulus
    eeg_epochs[f] = {stim_label: loaded_data[stim_label] for stim_label in loaded_data.files}

    # Import settings
    with open(f"Data\\Masters_testing\\{file}.json", "r") as file_object:
        settings[f] = json.load(file_object)

    # Import RS eyes open data
    rs_open_data[f] = np.load(f"Data\\Masters_testing\\{file}_open.npy", allow_pickle=True)

# Calculate PSD of all Epochs

In [3]:
# PSD settings
window_size = 10  # 10 = 0.1 Hz resolution, 5 = 0.2 Hz resolution, 2 = 0.5 Hz resolution

# Preallocate variables
eeg_f = [None] * len(files)
eeg_pxx = [None] * len(files)  # Preallocate to list in case not all files have the same number of channels

# Compute PSD for each file
for f, file in enumerate(files):
    eeg_f[f] = {}
    eeg_pxx[f] = {}

    # Compute PSD for each stimulus
    for stim_label, epochs in eeg_epochs[f].items():
        eeg_f[f][stim_label] = []
        eeg_pxx[f][stim_label] = []

        # Compute PSD for each epoch
        for epoch in epochs:
            f_values, pxx_values = signal.welch(
                x=epoch,
                fs=settings[f]["eeg_srate"],
                nperseg=window_size * settings[f]["eeg_srate"],
                noverlap= (window_size * settings[f]["eeg_srate"]) * 0.5,  # 50% overlap between windows
            )
            eeg_f[f][stim_label].append(f_values)
            eeg_pxx[f][stim_label].append(pxx_values)

        # Convert lists to arrays for consistency
        eeg_f[f][stim_label] = np.array(eeg_f[f][stim_label])
        eeg_pxx[f][stim_label] = np.array(eeg_pxx[f][stim_label])

# Compute SNR for all Epochs
- SNR is calculated for each epoch and then averaged per stimulus

In [4]:
# Settings
noise_band = 1    # Single-sided noise band [Hz]
nharms = 1        # Number of harmonics used
db_out = True     # Boolean to get output in dB
stim_freq = 10.0  # Example frequency, replace with your actual frequency

# Preallocate variables
snr = [None] * len(files)
epoch_count_snr = {}
epochs_snr = []

# Determine the number of epochs for each stimulus
for stim_idx, stim_label in settings[0]["stimuli"].items():  # Get labels from settings
    epoch_count_snr[stim_label] = eeg_pxx[0][stim_label].shape[0]

for f0, _ in enumerate(files):
    stim_labels = list(settings[f0]["stimuli"].values())
    temp_snr = np.zeros([len(stim_labels), len(settings[f0]["new_ch_names"])])

    # Compute SNR per stimulus
    for stim_idx, stim_label in settings[f0]["stimuli"].items():
        s = stim_labels.index(stim_label)  # Get index based on order in settings[f0]["stimuli"]

        for epoch in range(epoch_count_snr[stim_label]):
            snr_value = processing.ssvep_snr(
                f=eeg_f[f0][stim_label][epoch],  
                pxx=eeg_pxx[f0][stim_label][epoch, :, :], 
                stim_freq=stim_freq,
                noise_band=noise_band,
                nharms=nharms,
                db_out=db_out
            )

            epochs_snr.append(snr_value)

        temp_snr[s,:] = np.mean(np.array(epochs_snr), axis=0)

# Save temp SNR value
snr[f0] = temp_snr

# Export SNR

In [ ]:
save_snr = False   # Boolean to save SNRs to CSV

# Preallocate empty list to store all dataFrames
dfs = []

for f0, file in enumerate(files):
    # Preallocate variables
    col_names = []

    # Get the shape of the snr matrix for the current file
    snr_shape = snr[f0].shape
    temp_snr = np.zeros((len(settings[f0]['new_ch_names']), len(settings[f0]["stimuli"])))

    col_idx = 0
    # Loop through stimuli to fill the SNR matrix
    for s, stimuli in settings[f0]["stimuli"].items():
        if int(s) < snr_shape[0]:
            temp_snr[:, col_idx] = snr[f0][int(s), :]
            col_names.append(f"{stimuli}")
            col_idx += 1
        else:
            print(f"Stimulus index {s} is out of bounds for snr[f0].shape[0]: {snr_shape[0]}")

    # Prepare the row names using the new channel names
    row_names = [f"{file.split('_')[0]} - {channel}" for channel in settings[f0]['new_ch_names']]

    # Create DataFrame for the file
    dfs.append(
        pd.DataFrame(
            data=temp_snr,  
            columns=col_names,
            index=row_names
        )
    )

# Concatenate all DataFrames
snr_df = pd.concat(dfs)

# Save SNRs to CSV
if save_snr:
    snr_df.to_csv("Data\\Masters_testing\\snr_results_test2.csv")
